In [ ]:
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install pandas --quiet
!pip install datetime --quiet

**Imports**

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup

**Functions**

In [2]:
def read_metadata(filename):
    data = pd.read_csv(filename).drop(columns = 'Unnamed: 0')
    data['Content'] = None
    return data

def get_url(cellar_ref, doctype="03"):
    psid = cellar_ref
    psname = "cellar" # other options: cellar, celex, oj...
    lancode = "0006" # language code
    doctype = doctype # default: 03
    docnum = "DOC_1"
    # for further information, see Documentation Page 37: https://op.europa.eu/en/publication-detail/-/publication/50ecce27-857e-11e8-ac6a-01aa75ed71a1/language-en/format-PDF/source-73059305
    return f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"

def get_content(URL):
    response = requests.get(URL, headers={"Accept-Language":"en-US"})
    # one minor bug still in there: some requests (for example number 58 in 20220601_larger_data_b) are a valid request but have to download many mb first. the solution would be to stop the request.get if it runs longer than x seconds
    try:
        soup = BeautifulSoup(response.content, "html.parser")
        if str(soup)[1:4] == "PDF":
            '''
            in some (few) cases, the doctype is not 03 but 02. change it for these cases
            '''
            print("pdf detected, but fixed")
            doctype = '02'
            URL = URL[:-8] + doctype + URL[-6:]
            response = requests.get(URL, headers={"Accept-Language":"en-US"})
            soup = BeautifulSoup(response.content, "html.parser")
        else:
            print("no problem here")
            doctype = '03'
    except:
        '''
        in case there is an error
        '''
        print("yes problem here")
        URL = URL[:-8] + '02' + URL[-6:]
        response = requests.get(URL, headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
        
    if soup.find("p", class_="oj-normal") == None:
        content = ' '.join([item.text for item in soup.find_all("p", class_="normal")])
    else:
        content = ' '.join([item.text for item in soup.find_all("p", class_="oj-normal")])
    return content

def clean_data(data):
    data = data[data['Content'] != ""]
    data = data[data['Content'].str[0:3] == 'THE'] #remove content in other languages
    data = data[data['Content'].str.contains('Whereas: ')] # contains the split word
    data.loc[:, 'Content'] = data['Content'].apply(lambda x: x.split('Whereas: ', 1)[1]) # split off header
    data = data[data['Content'].str[0:3] == "(1)"] #gotta make sure it's standardized!
    return data.reset_index().drop(columns = "index")

def get_all_content(data):
    cellar_references = data['cellar']    
    for index, ref in enumerate(cellar_references):
        data.loc[index, 'Content'] = get_content(get_url(ref))
        print(f'Row {index} with cellar-number {ref} done')
    return data

**Workflow**

In [3]:
%%time
#retrieve metadata
filename = "../raw_data/20220602.csv"
metadata = read_metadata(filename)

# subset metadata
metadata_subset = metadata.iloc[6851:]

# get content
data_with_content = get_all_content(metadata)#_subset)

# clean content
data_with_content_clean = clean_data(data_with_content8)

# export data to csv
#data_b_with_content.to_csv("../raw_data/20220601_larger_data_b_scraped.csv")

no problem here
Row 0 with cellar-number 3cb645b2-2a23-43c9-b842-b03665a6733a done
no problem here
Row 1 with cellar-number 3b729ddf-f1f7-11e3-8cd4-01aa75ed71a1 done
no problem here
Row 2 with cellar-number 8917d52e-4432-11e2-9b3b-01aa75ed71a1 done


/home/jakobg/.pyenv/versions/3.8.12/envs/BatchLegal/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


no problem here
Row 3 with cellar-number ff333821-8325-4fcd-aad0-0a6f3913a242 done
no problem here
Row 4 with cellar-number e351eb07-9713-11e9-9369-01aa75ed71a1 done
no problem here
Row 5 with cellar-number 3cc00bdc-955e-11e3-8c34-01aa75ed71a1 done


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


pdf detected, but fixed
Row 6 with cellar-number fa9532bb-12e6-11e8-9253-01aa75ed71a1 done
no problem here
Row 7 with cellar-number 4763e956-79a1-42db-851f-c9521271ad5c done
no problem here
Row 8 with cellar-number e0c803af-9e0f-11e4-872e-01aa75ed71a1 done
no problem here
Row 9 with cellar-number 61d76918-8b7a-11eb-b85c-01aa75ed71a1 done
no problem here
Row 10 with cellar-number b3c20c11-85b9-11e4-b8a5-01aa75ed71a1 done
no problem here
Row 11 with cellar-number 2736e514-b31b-11eb-8aca-01aa75ed71a1 done
no problem here
Row 12 with cellar-number 1b011b40-2003-11eb-b57e-01aa75ed71a1 done
no problem here
Row 13 with cellar-number 6bf01c54-d572-4952-beaa-159de150f119 done
no problem here
Row 14 with cellar-number de093874-a22d-40ef-a7bf-5998d8920152 done


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


pdf detected, but fixed
Row 1185 with cellar-number 3ac2ed56-0475-11eb-a511-01aa75ed71a1 done
no problem here
Row 1186 with cellar-number b46df43a-71c6-11eb-9ac9-01aa75ed71a1 done
no problem here
Row 1187 with cellar-number 1906f664-27e6-11eb-9d7e-01aa75ed71a1 done
no problem here
Row 1188 with cellar-number 914d5e9e-a568-11ea-bb7a-01aa75ed71a1 done
no problem here
Row 1189 with cellar-number a30fd9d8-cd73-11eb-ac72-01aa75ed71a1 done
no problem here
Row 1190 with cellar-number bf6624ef-e4d5-11e9-9c4e-01aa75ed71a1 done
no problem here
Row 1191 with cellar-number fbe1003c-9fd7-11e5-8781-01aa75ed71a1 done
no problem here
Row 1192 with cellar-number 1f17ab2e-2fcf-11e7-9412-01aa75ed71a1 done
no problem here
Row 1193 with cellar-number d691bb22-983b-11ec-b4e4-01aa75ed71a1 done
no problem here
Row 1194 with cellar-number a483c535-8750-11e4-b8a5-01aa75ed71a1 done
no problem here
Row 1195 with cellar-number e12b2fa5-2f6e-11e5-9f85-01aa75ed71a1 done
no problem here
Row 1196 with cellar-number 94

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


pdf detected, but fixed
Row 1228 with cellar-number 398b4d22-0474-11eb-a511-01aa75ed71a1 done
no problem here
Row 1229 with cellar-number a974b890-424a-11e5-9f5a-01aa75ed71a1 done
no problem here
Row 1230 with cellar-number c76d1ce8-f783-11e3-831f-01aa75ed71a1 done
no problem here
Row 1231 with cellar-number 0e48eb0a-9d00-11e7-b92d-01aa75ed71a1 done
no problem here
Row 1232 with cellar-number 0d1f031a-8564-11e8-ac6a-01aa75ed71a1 done
no problem here
Row 1233 with cellar-number 1287754d-dc8f-11e6-ad7c-01aa75ed71a1 done
no problem here
Row 1234 with cellar-number a974594e-71c5-11e7-b2f2-01aa75ed71a1 done
no problem here
Row 1235 with cellar-number 6e43713e-ce6d-11e7-a5d5-01aa75ed71a1 done
no problem here
Row 1236 with cellar-number 844c308c-45b7-11ec-89db-01aa75ed71a1 done
no problem here
Row 1237 with cellar-number 0c001d4e-36da-441b-bd68-e78d4e40f360 done
no problem here
Row 1238 with cellar-number bea818f1-4d1b-11e3-ae03-01aa75ed71a1 done
no problem here
Row 1239 with cellar-number bf

KeyboardInterrupt: 

In [4]:
data_with_content

NameError: name 'data_with_content' is not defined

In [11]:
len(metadata)

23960

**Read already processed Data**

In [ ]:
# data_with_content: Zeile 0 - 1242 (what's up with that?). 
# data_with_content3: Zeile 2833 - 3855
# data_with_content4: Zeile 3855 - 4721
# data_with_content5: Zeile 4721 - 5147
# data_with_content6: Zeile 5147 - 6851
# data_with_content8: Zeile 6851 - 8839


In [ ]:
data = pd.read_csv("../raw_data/20220601_larger_data_b_scraped_clean.csv")
data = clean_data(data)

In [26]:
len(data_with_content8)

NameError: name 'data_with_content8' is not defined

In [ ]:
data.to_csv("../raw_data/20220601_larger_data_b_scraped_clean_v3.csv")

In [ ]:
get_url(data['cellar'][100])

**Test Area**

In [ ]:
metadata.iloc[3498:3502]

In [ ]:
data = read_metadata(filename)

In [ ]:
URL = get_url(metadata['cellar'][58], "03")

In [ ]:
response = requests.get(URL, headers={"Accept-Language":"en-US"})
try:
    soup = BeautifulSoup(response.content, "html.parser")
    if str(soup)[1:4] == "PDF":
        print("pdf detected, but fixed")
        '''
        in some (few) cases, the doctype is not 03 but 02. change it for these cases
        '''
        URL = URL[:-8] + '02' + URL[-6:]
        response = requests.get(URL, headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
    else:
        print("no problem here")
except:
    '''
    in case there is an error
    '''
    print("yes problem here")
    URL = URL[:-8] + '02' + URL[-6:]
    response = requests.get(URL, headers={"Accept-Language":"en-US"})
    soup = BeautifulSoup(response.content, "html.parser")

if soup.find("p", class_="oj-normal") == None:
    content = ' '.join([item.text for item in soup.find_all("p", class_="normal")])
else:
    content = ' '.join([item.text for item in soup.find_all("p", class_="oj-normal")])

print(URL)

content#.split('Whereas:', 1)[1] # only return text without the head